<a href="https://colab.research.google.com/github/SelmaDM/Pyterrier/blob/master/PyTerrier-Neural%20Re-ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTerrier - Neural Re-Ranking 

Dans ce TP vous allez :

 - reclasser des documents en utilisant des modèles neuronaux comme KNRM, Vanilla BERT, EPIC et monoT5.


# Installation


In [ ]:
!pip install -q --upgrade python-terrier

### Installation des plugins Pyterrier  

Nous installons les plugins PyTerrier [OpenNIR](https://opennir.net/) et [monoT5](https://github.com/terrierteam/pyterrier_t5). Vous pouvez ignorer sans risque les erreurs de version des paquets.

In [ ]:
!pip install -q --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install -q --upgrade git+https://github.com/terrierteam/pyterrier_t5

## Preliminary steps

Ces lignes sont nécessaires pour travailler avec DeepCT.

In [ ]:
%tensorflow_version 1.x
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
assert tf.__version__.startswith("1"), "TF 1 is required by DeepCT; on Colab, use %tensorflow_version 1.x"
tf.logging.set_verbosity(tf.logging.ERROR)

TensorFlow 1.x selected.


**[PyTerrier](https://github.com/terrier-org/pyterrier) initialization** 

In [ ]:
import pyterrier as pt
if not pt.started():
    pt.init()

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')


from pyterrier.measures import * # allow for natural measure names
import onir_pt

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

[INFO] trec-robust04 is deprecated. Consider using disks45/nocr/trec-robust-2004 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold1 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold1 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold2 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold2 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold3 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold3 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold4 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold4 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold5 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold5 instead, which provides better parsing of the corpus.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


### Indexation

In [ ]:
!rm -rf ./terrier_cord19/*

In [ ]:
import os

pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer 
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

cord19/trec-covid documents:   0%|          | 0/192509 s<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  del sys.path[0]


21:07:24.286 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed
21:08:58.801 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 54937 empty documents
21:09:00.248 [ForkJoinPool-1-worker-3] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BlockIndexer.createDirectIndex(BlockIndexer.java:472)
	at org.terrier.structures.indexing

## Re-Rankers 

Commençons à explorer quelques méthodes neuronales de re-classement ! Nous pouvons les construire à partir de zéro en utilisant `onir_pt.reranker`.

Le modèle de re-ranking d'OpenNIR est composé de :
 - `ranker` (par exemple, `drmm`, `knrm`, ou `pacrr`). Ceci définit l'architecture neuronale de classement.
 - `vocab` (par exemple, `wordvec_hash`, ou `bert`). Ceci définit comment le texte est encodé par le modèle. Cette approche rend facile l'échange de différentes représentations de texte.

L'exécution de cette ligne prendra quelques minutes car elle télécharge et prépare les vecteurs de mots.

In [ ]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')

[2022-04-03 21:34:14,509][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2022-04-03 21:34:24,751][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [10.24s]


Voyons comment ces modèles fonctionnent pour le classement !

In [ ]:
tfidf = pt.BatchRetrieve(index_ref, wmodel="TF_IDF") % 50
get_text = pt.text.get_text(cord19, 'abstract') #>> pt.apply.title_abstract(lambda r: r['title'] + ' ' + r['abstract'])


In [ ]:
topics = cord19.get_topics(variant='description')
qrels = cord19.get_qrels()

/usr/local/lib/python3.7/dist-packages/pyterrier/datasets.py:433: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)


In [ ]:
SEED=42

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=SEED)


test_qrels = qrels # seulement les annotations des topics en réponse sont utilisés, donc pas de problème si on utilise tout
train_qrels = qrels
valid_qrels = qrels

In [ ]:

# build a sub-pipeline to get the concatenated title and abstract text
pipeline = tfidf >> get_text >> knrm
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> KNRM'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

[2022-04-03 21:09:35,827][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2022-04-03 21:09:35,839][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/188 s<?, ?it/s]

[2022-04-03 21:09:36,988][onir_pt][DEBUG] [finished] batches: [1.15s] [188it] [163.84it/s]


,name,AP(rel=2),nDCG,nDCG@10,P(rel=2)@10
0,TFIDF,0.054568,0.123589,0.595818,0.546667
1,TFIDF >> KNRM,0.042547,0.112703,0.423958,0.360000


Cela ne fonctionne pas très bien car le modèle n'est pas entraîné ; il utilise des poids aléatoires pour combiner les scores de la matrice de similarité.

## Entraînement du re-ranker

Vous pouvez entraîner des modèles de re-classement dans PyTerrier en utilisant la méthode `fit`. 

In [ ]:
pipeline.fit(
    train_topics,
    train_qrels,
    valid_topics,
    valid_qrels)

[2022-04-03 21:30:05,863][onir_pt][DEBUG] [starting] validation
[2022-04-03 21:30:05,866][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2022-04-03 21:30:05,868][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/63 s<?, ?it/s]

[2022-04-03 21:30:06,242][onir_pt][DEBUG] [finished] batches: s] [63it] [169.11it/s]
[2022-04-03 21:30:06,435][onir_pt][DEBUG] [finished] validation s]
[2022-04-03 21:30:06,436][onir_pt][INFO] pre-validation: 0.0061
[2022-04-03 21:30:06,439][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2022-04-03 21:30:06,440][onir_pt][DEBUG] [starting] training
[2022-04-03 21:30:06,441][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2022-04-03 21:30:06,815][onir_pt][DEBUG] not enough negs
[2022-04-03 21:30:07,828][onir_pt][DEBUG] not enough negs
[2022-04-03 21:30:09,827][onir_pt][DEBUG] not enough negs
[2022-04-03 21:30:11,892][onir_pt][DEBUG] not enough negs
[2022-04-03 21:30:12,785][onir_pt][DEBUG] not enough negs
[2022-04-03 21:30:14,794][onir_pt][DEBUG] not enough negs
[2022-04-03 21:30:16,861][onir_pt][DEBUG] [finished] train pairs: [10.42s] [1024it] [98.28it/s]
[2022-04-03 21:30:16,867][onir_pt][DEBUG] [finished] training [10.43s]
[2022-04-03 21:30:16,868][onir_pt][INFO] training   it=0 loss=0.1864
[2022-04-03 21:30:16,869][onir_pt][DEBUG] [starting] validation
[2022-04-03 21:30:16,873][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2022-04-03 21:30:16,874][onir_pt][DEBUG] [starting] batches


...

[2022-04-03 21:33:59,544][onir_pt][DEBUG] [finished] batches: s] [63it] [139.94it/s]
[2022-04-03 21:33:59,741][onir_pt][DEBUG] [finished] validation s]
[2022-04-03 21:33:59,742][onir_pt][INFO] validation it=20 map=0.0061 ndcg=0.0155 P_10=0.0700
[2022-04-03 21:33:59,742][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2022-04-03 21:33:59,743][onir_pt][DEBUG] [starting] training
[2022-04-03 21:33:59,744][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2022-04-03 21:34:00,285][onir_pt][DEBUG] not enough negs
[2022-04-03 21:34:01,434][onir_pt][DEBUG] not enough negs
[2022-04-03 21:34:01,814][onir_pt][DEBUG] not enough negs
[2022-04-03 21:34:02,540][onir_pt][DEBUG] not enough negs
[2022-04-03 21:34:05,119][onir_pt][DEBUG] not enough negs
[2022-04-03 21:34:05,756][onir_pt][DEBUG] not enough negs
[2022-04-03 21:34:07,138][onir_pt][DEBUG] not enough negs
[2022-04-03 21:34:07,941][onir_pt][DEBUG] not enough negs
[2022-04-03 21:34:08,474][onir_pt][DEBUG] not enough negs
[2022-04-03 21:34:10,755][onir_pt][DEBUG] [finished] train pairs: [11.01s] [1024it] [93.00it/s]
[2022-04-03 21:34:10,760][onir_pt][DEBUG] [finished] training [11.02s]
[2022-04-03 21:34:10,761][onir_pt][INFO] training   it=21 loss=0.1850
[2022-04-03 21:34:10,762][onir_pt][DEBUG] [starting] validation
[2022-04-03 21:34:10,762][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2022-04-03 21:34:10,763][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/63 s<?, ?it/s]

[2022-04-03 21:34:11,196][onir_pt][DEBUG] [finished] batches: s] [63it] [146.16it/s]
[2022-04-03 21:34:11,377][onir_pt][DEBUG] [finished] validation s]
[2022-04-03 21:34:11,378][onir_pt][INFO] validation it=21 map=0.0061 ndcg=0.0154 P_10=0.0680
[2022-04-03 21:34:11,379][onir_pt][INFO] early stopping; model reverting back to it=1


In [ ]:
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> KNRM (trained)'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

[2022-04-03 21:34:13,522][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2022-04-03 21:34:13,523][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/188 s<?, ?it/s]

[2022-04-03 21:34:14,426][onir_pt][DEBUG] [finished] batches: s] [188it] [208.41it/s]


,name,AP(rel=2),nDCG,nDCG@10,P(rel=2)@10
0,TFIDF,0.054568,0.123589,0.595818,0.546667
1,TFIDF >> KNRM (trained),0.053241,0.122879,0.573381,0.493333


Le résultat est plus élévé, mais cela reste moins bon que le TFIDF. Proposez une hypothèse sur le problème.

## Vanilla BERT

Les modèles linguistiques contextualisés, tels que [BERT] (https://arxiv.org/abs/1810.04805), sont des modèles neuronaux beaucoup plus puissants qui se sont avérés efficaces pour le classement.

Nous allons essayer d'utiliser une version "vanille" (ou "mono") du modèle BERT. Le modèle BERT est pré-entraîné pour la modélisation du langage et la prédiction de la phrase suivante.

In [ ]:
#del knrm # clear out memory from KNRM
vbert = onir_pt.reranker('vanilla_transformer', 'bert', text_field='abstract', vocab_config={'train': True})

Voyons comment ce modèle se comporte sur TREC COVID.

In [ ]:
pipeline = tfidf % 50 >> get_text >> vbert
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> VBERT'],
    baseline=0,
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

Comme nous le constatons, bien que le modèle soit pré-entraîné, il n'obtient pas de très bons résultats. Cela est dû au fait qu'il n'est pas réglé pour la tâche de classement par pertinence.

Cependant, nous pouvons entraîner le modèle pour le classement (comme indiqué ci-dessus pour KNRM).

## monoT5

Le modèle [monoT5](https://arxiv.org/abs/2003.06713) évalue les documents à l'aide d'un modèle de langage causal. Voyons comment cette approche fonctionne sur TREC COVID.

La classe `MonoT5ReRanker` de `pyterrier_t5` charge automatiquement une version du classeur monoT5 qui est entraînée sur le jeu de données MS MARCO passage.

In [ ]:
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker(text_field='abstract')

In [ ]:
pipeline = (tfidf >> get_text >> monoT5)
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> T5'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10, "mrt"]
)

Comme on pouvait s'y attendre, les résultats sont bien meilleurs en termes de NDCG@10 (0.5958 vs 0.6855).

# Tâche pratique

Comme pour le TP precedent, utilisez les modèles implémentés pour cord19 dans une tâche de question-réponse. Dans ce contexte, les requêtes sont de questions et les documents sont des documents qui pourraient contenir la réponse. Notez que vous devez refaire l'indexation ainsi que les autres étapes étudiées dans ce TP. Vous pouvez le télécharger le dataset en utilisant les lines de code ci-dessous.

In [ ]:
fiqa = {}
fiqa['train'] = pt.datasets.get_dataset('irds:beir/fiqa/train')
fiqa['valid'] = pt.datasets.get_dataset('irds:beir/fiqa/dev')
fiqa['test'] = pt.datasets.get_dataset('irds:beir/fiqa/test')

test_topics = fiqa['test'].get_topics(variant='text')
test_qrels = fiqa['test'].get_qrels()

train_topics = fiqa['train'].get_topics(variant='text')
train_qrels = fiqa['train'].get_qrels()

valid_topics = fiqa['valid'].get_topics(variant='text')
valid_qrels = fiqa['valid'].get_qrels()

In [ ]:

#INDEXATION

import pyterrier as pt
if not pt.started():
  pt.init()
fiqa = pt.datasets.get_dataset('irds:beir/fiqa')

import os

pt_index_path = './terrier_fiqa'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer 
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(fiqa.get_corpus_iter(), 
                             )

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
    

[INFO] [starting] building docstore
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]
docs_iter: 100%|████████████████████████| 57638/57638 [2.51s<0ms, 22986.49doc/s]
[INFO] [finished] docs_iter: [2.51s] [57638doc] [22972.84doc/s]
[INFO] [finished] building docstore [2.52s]


beir/fiqa documents:   0%|          | 0/57638 s<?, ?it/s]

In [ ]:
tfidf = pt.BatchRetrieve(index_ref, wmodel="TF_IDF") % 50
get_text = pt.text.get_text(fiqa, 'abstract') #>> pt.apply.title_abstract(lambda r: r['title'] + ' ' + r['abstract'])

In [ ]:
topics = fiqa.get_topics(variant='description')
qrels = fiqa.get_qrels()

In [ ]:

# build a sub-pipeline to get the concatenated title and abstract text
pipeline = tfidf >> get_text >> knrm
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> KNRM'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

In [ ]:
pipeline.fit(
    train_topics,
    train_qrels,
    valid_topics,
    valid_qrels)

In [ ]:
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> KNRM (trained)'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)


Identifiez-vous des avantages par rapport à l'utilisation de Learning to Rank ?